In [1]:
import pandas as pd
import numpy as np
import dill
dill._dill._reverse_typemap['ClassType'] = type
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
import warnings
warnings.simplefilter('ignore')

Загрузим модель и тестовые данные и проверим работу модели

In [2]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [3]:
X_test.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,8879-ZKJOF,Female,0,No,No,41,Yes,No,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),79.85,3320.75
1,0201-MIBOL,Female,1,No,No,66,Yes,Yes,Fiber optic,Yes,No,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),102.40,6471.85
2,1600-DILPE,Female,0,No,No,12,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),45.00,524.35


In [4]:
with open('catboost_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [5]:
predictions = pipeline.predict_proba(X_test)
pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

In [6]:
precision, recall, thresholds = precision_recall_curve(y_test, predictions[:, 1][:])
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, Roc_auc=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix], roc_auc_score(y_test, predictions[:, 1][:])))  

Best Threshold=0.366063, F-Score=0.646, Precision=0.569, Recall=0.746, Roc_auc=0.863


Результат такой же, как на Step 1.

Попробуем дать модели новые данные с пропусками и ошибками

In [7]:
customerID, gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines, InternetService, OnlineSecurity, \
OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, \
PaymentMethod, MonthlyCharges, TotalCharges = ('', 'lkj', 0, 'No', 'No', 3, 'Yes', 'No', 'DSL', \
                                               'Yes', 'No', 'aaa', '', '', '', '', 'Yes',\
                                               '','dfg', '987')

In [8]:
pipeline.predict_proba(pd.DataFrame({'customerID': [customerID],
                                          'gender': [gender],
                                          'SeniorCitizen': [SeniorCitizen],
                                          'Partner': [Partner],
                                          'Dependents': [Dependents],
                                          'tenure': [tenure],
                                          'PhoneService': [PhoneService],
                                          'MultipleLines': [MultipleLines],
                                          'InternetService': [InternetService],
                                          'OnlineSecurity': [OnlineSecurity],
                                          'OnlineBackup': [OnlineBackup],
                                          'DeviceProtection': [DeviceProtection],
                                          'TechSupport': [TechSupport],
                                          'StreamingTV': [StreamingTV],
                                          'StreamingMovies': [StreamingMovies],
                                          'Contract': [Contract],
                                          'PaperlessBilling': [PaperlessBilling],
                                          'PaymentMethod': [PaymentMethod],
                                          'MonthlyCharges': [MonthlyCharges],
                                          'TotalCharges': [TotalCharges]}))

array([[0.84439982, 0.15560018]])